In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
import json
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
from src.cmos_plotter.Latency_calculator import *
from src.utils.logger_functions import console
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [ ]:
MAIN_PATH = '...'
PROCESSED_DATA_PATH = os.path.join(MAIN_PATH, f'Processed_data/')
OUTPUT_PATH = os.path.join(MAIN_PATH,f'...')
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

filename = '...'

SORTER_PATH = PROCESSED_DATA_PATH
FULL_SORTER_PATH = os.path.join(SORTER_PATH,'Sorter_'+filename)

PAIRINGS_PATH = os.path.join(MAIN_PATH, f'...')

In [ ]:
data = np.load(os.path.join(PROCESSED_DATA_PATH, f'{filename[:-3]}_processed.pkl'), allow_pickle=True)
spikes_extremum = pd.DataFrame(data['SPIKEMAT_EXTREMUM'])
spikes = np.array([[int(row[0]), float(row[1]), float(row[2])] for row in data['SPIKEMAT']])
a = 0
area = filename.split('_')[1]
data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
if 'validated_results' in data_te.keys():
    console.info(f'Pairs from {filename} were already validated')
if a ==0:
    exp_duration = data_te['EXPERIMENT_DURATION']
    te_unit_pairs = pd.DataFrame(data_te['mTE'])
    pairings = te_unit_pairs
    

    with open(os.path.join(FULL_SORTER_PATH, 'wf_folder_curated/sparsity.json'), 'r') as file:
        sorting_info = json.load(file)

    unit_ids = sorting_info['unit_ids']

    spikes, electrodes_pre_all, electrodes_post_all, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all = get_electrode_unit_info_te(data, pairings, area, unit_ids)

In [ ]:

for idx in range(len(unit_pre_all)):
    unit_pre = unit_pre_all[idx]
    unit_post = unit_post_all[idx]
    pre_extremum = pre_extremum_all[idx]
    post_extremum = post_extremum_all[idx]
    lag = lag_all[idx]
    unit_to_el = data_te['UNIT_TO_EL']

    lat = plot_latency_and_location_with_extremum_both_plots(OUTPUT_PATH, filename, lag, exp_duration, unit_to_el, pre_extremum, post_extremum, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output = post_extremum)

In [ ]:
lat = plot_latency_and_location_with_extremum_both_plots(OUTPUT_PATH, filename, lag, exp_duration, unit_to_el, pre_extremum, post_extremum, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output = post_extremum)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
plt.subplots_adjust(left=0., right=0.95, bottom=0.15, top=0.95)

input_color = '#2bace2'
output_color = '#f281b2'


input = lat[lat['category'] == 'input']
output = lat[lat['category'] == 'output']



plt.scatter(output['latency'], output['spike time'] / 1000, s=1, color=output_color, alpha=0.8)
plt.scatter(input['latency'], input['spike time'] / 1000, s=1, color=input_color, alpha=0.8)
plt.xlabel('Latency [ms]', fontsize=12)
plt.ylabel('Experiment Time [s]', fontsize=12)
plt.ylim(-0.5, 100)
plt.xlim(-0.5, 6)

ax.spines['left'].set_position(('outward', 5))
ax.spines['bottom'].set_position(('outward', 5))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)


plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_before_stimulation'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_before_stimulation.pdf'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_before_stimulation.svg'), dpi=300, bbox_inches='tight')


# Show plot
plt.show()

In [ ]:
# Create figure and axes with specific padding
fig, ax = plt.subplots(figsize=(4,3))

latency_week_after = lat_after

# Set space between axes and plot area
plt.subplots_adjust(left=0., right=0.95, bottom=0.15, top=0.95)

input_color = '#2bace2'
output_color = '#f281b2'

# Plot latency_before data

input_week_after = latency_week_after[latency_week_after['category'] == 'input']
output_week_after = latency_week_after[latency_week_after['category'] == 'output']

# Use plt.scatter directly


plt.scatter(output_week_after['latency'], output_week_after['spike time'] / 1000, s=1, color=output_color, alpha=0.8)
plt.scatter(input_week_after['latency'], input_week_after['spike time'] / 1000, s=1, color=input_color, alpha=0.8)
# Set labels
plt.xlabel('Latency [ms]', fontsize=12)
plt.ylabel('Experiment Time [s]', fontsize=12)
plt.ylim(-0.50,100)
plt.xlim(-0.5, 6)


# Adjust spines to create space between axes and plot area
ax.spines['left'].set_position(('outward', 5))
ax.spines['bottom'].set_position(('outward', 5))

# Turn off the top and right spines to match your example
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
#plt.xticks(np.arange(0, 11, 2), fontsize = 12)
#plt.yticks(np.arange(0, 400, 100), fontsize = 12)

# Make the layout compact
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_after_stimulation'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_after_stimulation.pdf'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_after_stimulation.svg'), dpi=300, bbox_inches='tight')
# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# I'm assuming you already have your three arrays: latency_before, latency_after, and latency_after_2

# Create figure and axes with specific padding
fig, ax = plt.subplots(figsize=(4,8))

# Set space between axes and plot area
plt.subplots_adjust(left=0., right=0.95, bottom=0.15, top=0.95)

input_color = '#2bace2'
output_color = '#f281b2'

# Plot latency_before data
input_before = lat_before[lat_before['category'] == 'input']
output_before = lat_before[lat_before['category'] == 'output']

# Use plt.scatter directly
plt.scatter(input_before['latency'], input_before['spike time'] / 1000, s=7, color=input_color, alpha=0.8)
plt.scatter(output_before['latency'], output_before['spike time'] / 1000, s=7, color=output_color, alpha=0.8)

# Plot latency_after data (add 5 min = 300000 ms)
input_after = lat_after[lat_after['category'] == 'input']
output_after = lat_after[lat_after['category'] == 'output']

plt.scatter(input_after['latency'], input_after['spike time'] / 1000+600, s=7, color=input_color, alpha=0.8)
plt.scatter(output_after['latency'], output_after['spike time'] / 1000+600, s=7, color=output_color, alpha=0.8)



# Set x-axis ticks to show only even numbers (0,2,4,6,8,10)
plt.xticks(np.arange(0, 11, 2), fontsize = 18)


# Set axis limits based on your data (mirroring your example image)
#plt.xlim(-0.5, 10.5)
#plt.ylim(-5, 105)

# Set labels
plt.xlabel('Latency (ms)', fontsize=16)
plt.ylabel('Experiment Time (s)', fontsize=16)

# Adjust spines to create space between axes and plot area
ax.spines['left'].set_position(('outward', 5))
ax.spines['bottom'].set_position(('outward', 5))

# Turn off the top and right spines to match your example
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

# Make the layout compact
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_{filename[:-3]}_pair.png'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_{filename[:-3]}_pair.svg'), dpi=300, bbox_inches='tight')

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# I'm assuming you already have your three arrays: latency_before, latency_after, and latency_after_2

# Create figure and axes with specific padding
fig, ax = plt.subplots(figsize=(4,8))

# Set space between axes and plot area
plt.subplots_adjust(left=0., right=0.95, bottom=0.15, top=0.95)

input_color = '#2bace2'
output_color = '#f281b2'

# Plot latency_before data
input_before = latency_before[latency_before['category'] == 'input']
output_before = latency_before[latency_before['category'] == 'output']

# Use plt.scatter directly
plt.scatter(input_before['latency'], input_before['spike time'] / 1000, s=7, color=input_color, alpha=0.8)
plt.scatter(output_before['latency'], output_before['spike time'] / 1000, s=7, color=output_color, alpha=0.8)

# Plot latency_after data (add 5 min = 300000 ms)
input_after = latency_after[latency_after['category'] == 'input']
output_after = latency_after[latency_after['category'] == 'output']

plt.scatter(input_after['latency'], input_after['spike time'] / 1000+600, s=7, color=input_color, alpha=0.8)
plt.scatter(output_after['latency'], output_after['spike time'] / 1000+600, s=7, color=output_color, alpha=0.8)

# Plot latency_after_2 data (add 10 min = 600000 ms)
input_after_2 = latency_after_2[latency_after_2['category'] == 'input']
output_after_2 = latency_after_2[latency_after_2['category'] == 'output']

plt.scatter(input_after_2['latency'], input_after_2['spike time'] / 1000+900, s=7, color=input_color, alpha=0.8)
plt.scatter(output_after_2['latency'], output_after_2['spike time'] / 1000+900, s=7, color=output_color, alpha=0.8)

# Set x-axis ticks to show only even numbers (0,2,4,6,8,10)
plt.xticks(np.arange(0, 11, 2), fontsize = 18)


# Set axis limits based on your data (mirroring your example image)
#plt.xlim(-0.5, 10.5)
#plt.ylim(-5, 105)

# Set labels
plt.xlabel('Latency (ms)', fontsize=16)
plt.ylabel('Experiment Time (s)', fontsize=16)

# Adjust spines to create space between axes and plot area
ax.spines['left'].set_position(('outward', 5))
ax.spines['bottom'].set_position(('outward', 5))

# Turn off the top and right spines to match your example
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

# Make the layout compact
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_{filename[:-3]}_pair.png'), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(OUTPUT_PATH, f'Latency_plot_{filename[:-3]}_pair.svg'), dpi=300, bbox_inches='tight')

# Show plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# I'm assuming you already have your three arrays: latency_before, latency_after, and latency_after_2
# The code below shows how to plot them all on the same figure

fig, ax = plt.subplots(figsize=(4, 8))

# Colors for consistency
input_color = '#2bace2'
output_color = '#f281b2'

# Plot latency_before data
input_before = latency_before[latency_before['category'] == 'input']
output_before = latency_before[latency_before['category'] == 'output']
ax.scatter(output_before['latency'], output_before['spike time'] / 1000, s=7, color=output_color, alpha=0.7)
ax.scatter(input_before['latency'], input_before['spike time'] / 1000, s=7, color=input_color, alpha=0.7)

# Plot latency_after data (add 5 min = 300000 ms)
input_after = latency_after[latency_after['category'] == 'input']
output_after = latency_after[latency_after['category'] == 'output']
ax.scatter(output_after['latency'], output_after['spike time'] / 1000 + 600, s=7,  color=output_color, alpha=0.7)
ax.scatter(input_after['latency'], input_after['spike time'] / 1000 + 600, s=7, color=input_color, alpha=0.7)

# Plot latency_after_2 data (add 10 min = 600000 ms)
input_after_2 = latency_after_2[latency_after_2['category'] == 'input']
output_after_2 = latency_after_2[latency_after_2['category'] == 'output']
ax.scatter(output_after_2['latency'], output_after_2['spike time'] / 1000 + 900, s=7, color=output_color, alpha=0.7)
ax.scatter(input_after_2['latency'], input_after_2['spike time'] / 1000 + 900, s=7, color=input_color, alpha=0.7)

# Add horizontal lines to separate the periods
y_min, y_max = ax.get_ylim()
ax.axhline(y=300, color='gray', linestyle='--', alpha=0.5)  # Line at 5 min = 300 sec
ax.axhline(y=600, color='gray', linestyle='--', alpha=0.5)  # Line at 10 min = 600 sec

# Set labels and title
ax.set_xlabel('Latency (ms)')
ax.set_ylabel('Time (seconds)')
ax.set_title(f"Source {unit_pre} to Target {unit_post}")

# Create a custom legend to avoid duplicates
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))

# Add annotations to identify time periods
ax.text(ax.get_xlim()[1] * 0.95, 150, 'Before', horizontalalignment='right')
ax.text(ax.get_xlim()[1] * 0.95, 750, 'After', horizontalalignment='right')
ax.text(ax.get_xlim()[1] * 0.95, 1050, 'After_2', horizontalalignment='right')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def calculate_synaptic_probability(data, time_bin_size=30):
    """
    Calculate synaptic probability over time bins.
    
    Parameters:
    - data: Dictionary containing 'lag' and 'latency_extremum'
    - time_bin_size: Size of time bins in ms (default: 30)
    
    Returns:
    - time_bins: Array of time bin centers
    - probabilities: Array of synaptic probabilities for each time bin
    - prob_changes: Array of changes in probability between consecutive time bins
    """
    lag = data['lag']
    latency = data['latency_extremum']
    
    # Extract all spike times
    all_spike_times = latency['spike time']
    
    # Determine the range of time bins
    min_time = np.min(all_spike_times)
    max_time = np.max(all_spike_times)
    
    # Create time bins
    time_edges = np.arange(min_time, max_time + time_bin_size, time_bin_size)
    time_bins = (time_edges[:-1] + time_edges[1:]) / 2  # Centers of bins
    
    probabilities = []
    
    # For each time bin, calculate synaptic probability
    for i in range(len(time_bins)):
        bin_start = time_edges[i]
        bin_end = time_edges[i+1]
        
        # Filter latency data for current time bin
        time_mask = (all_spike_times >= bin_start) & (all_spike_times < bin_end)
        bin_latency = latency[time_mask]
        
        if len(bin_latency) == 0:
            # No spikes in this bin
            probabilities.append(0)
            continue
        
        # Filter for output spikes
        data_output = bin_latency[bin_latency['category'] == 'output']
        
        # Filter for latency within lag ± 1 ms
        latency_filtered = data_output[(data_output['latency'] >= lag - 1) & 
                                     (data_output['latency'] <= lag + 1)]
        
        # Get input spikes in this bin
        input_spikes = bin_latency[bin_latency['category'] == 'input']
        input_spike_counts = len(input_spikes)
        
        if input_spike_counts == 0:
            # No input spikes in this bin
            probabilities.append(0)
            continue
        
        # Count output spikes that correspond to input spikes
        output_spike_counts = np.sum(np.isin(latency_filtered['input spike'], 
                                          input_spikes['input spike']))
        
        # Calculate probability
        probability = output_spike_counts / input_spike_counts
        probabilities.append(probability)
    
    probabilities = np.array(probabilities)
    
    # Calculate changes in probability
    prob_changes = np.diff(probabilities, prepend=probabilities[0])
    
    return time_bins, probabilities, prob_changes

def plot_synaptic_probability(time_bins, probabilities, prob_changes):
    """Plot synaptic probability over time and its change."""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot synaptic probability over time
    ax1.plot(time_bins, probabilities, 'b-', linewidth=2)
    ax1.set_ylabel('Synaptic Probability')
    ax1.set_title('Synaptic Probability Over Time')
    ax1.grid(True)
    
    # Plot change in synaptic probability over time
    ax2.plot(time_bins, prob_changes, 'r-', linewidth=2)
    ax2.set_xlabel('Time (ms)')
    ax2.set_ylabel('Change in Probability')
    ax2.set_title('Change in Synaptic Probability Over Time')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()





In [ ]:
#res[31]['latency_extremum']
data = res[31]
# Calculate synaptic probability over time
time_bins, probabilities, prob_changes = calculate_synaptic_probability(data, time_bin_size=25000)

# Plot the results
plot_synaptic_probability(time_bins, probabilities, prob_changes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def calculate_synaptic_probability(latency, lag, time_bin_size=30):
    """
    Calculate synaptic probability over time bins.
    
    Parameters:
    - data: Dictionary containing 'lag' and 'latency_extremum'
    - time_bin_size: Size of time bins in ms (default: 30)
    
    Returns:
    - time_bins: Array of time bin centers (in seconds)
    - probabilities: Array of synaptic probabilities for each time bin
    - prob_changes: Array of changes in probability between consecutive time bins
    """
    # Extract all spike times
    all_spike_times = latency['spike time']
    
    # Determine the range of time bins
    min_time = np.min(all_spike_times)
    max_time = np.max(all_spike_times)
    
    # Create time bins
    time_edges = np.arange(min_time, max_time + time_bin_size, time_bin_size)
    time_bins = (time_edges[:-1] + time_edges[1:]) / 2  # Centers of bins
    
    # Convert time bins from ms to seconds
    time_bins_seconds = time_bins / 1000.0
    
    probabilities = []
    
    # For each time bin, calculate synaptic probability
    for i in range(len(time_bins)):
        bin_start = time_edges[i]
        bin_end = time_edges[i+1]
        
        # Filter latency data for current time bin
        time_mask = (all_spike_times >= bin_start) & (all_spike_times < bin_end)
        bin_latency = latency[time_mask]
        
        if len(bin_latency) == 0:
            # No spikes in this bin
            probabilities.append(0)
            continue
        
        # Filter for output spikes
        data_output = bin_latency[bin_latency['category'] == 'output']
        
        # Filter for latency within lag ± 1 ms
        latency_filtered = data_output[(data_output['latency'] >= lag - 1) & 
                                     (data_output['latency'] <= lag + 1)]
        
        # Get input spikes in this bin
        input_spikes = bin_latency[bin_latency['category'] == 'input']
        input_spike_counts = len(input_spikes)
        
        if input_spike_counts == 0:
            # No input spikes in this bin
            probabilities.append(0)
            continue
        
        # Count output spikes that correspond to input spikes
        output_spike_counts = np.sum(np.isin(latency_filtered['input spike'], 
                                          input_spikes['input spike']))
        
        # Calculate probability
        probability = output_spike_counts / input_spike_counts
        probabilities.append(probability)
    
    probabilities = np.array(probabilities)
    
    # Calculate changes in probability
    prob_changes = np.diff(probabilities, prepend=probabilities[0])
    
    return time_bins_seconds, probabilities, prob_changes

def plot_synaptic_probability(time_bins_seconds, probabilities, prob_changes):
    """Plot synaptic probability over time and its change."""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot synaptic probability over time
    ax1.plot(time_bins_seconds, probabilities, 'b-', linewidth=2)
    ax1.set_ylabel('Synaptic Probability')
    ax1.set_title('Synaptic Probability Over Time')
    ax1.grid(True)
    
    # Plot change in synaptic probability over time
    ax2.plot(time_bins_seconds, prob_changes, 'r-', linewidth=2)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Change in Probability')
    ax2.set_title('Change in Synaptic Probability Over Time')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()



In [ ]:
#data['latency_extremum']
#filter only cateogry output
latency = latency_after
latency = latency[latency['category'] == 'output']
#calculate mean latency
mean_latency = np.mean(latency['latency'])
median_latency = np.median(latency['latency'])
mean_latency, median_latency

In [ ]:
data_te.keys()

In [ ]:
all_probabilities = []
all_prob_changes = []
for row in data_te['validated_results']:
    if row['validation'] == 'good':
        latency = row['latency_extremum']
        lag = row['lag']
        time_bins, probabilities, prob_changes = calculate_synaptic_probability(latency, lag, time_bin_size=25000)
        all_probabilities.append(probabilities)
        all_prob_changes.append(prob_changes)

all_probabilities = np.array(all_probabilities)
all_prob_changes = np.array(all_prob_changes)
#plot mean of all probabilities
mean_probabilities = np.mean(all_probabilities, axis=0)
mean_prob_changes = np.mean(all_prob_changes, axis=0)
#plot mean of all probabilities
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
# Plot synaptic probability over time
ax1.plot(time_bins, mean_probabilities, 'b-', linewidth=2)
ax1.set_ylabel('Synaptic Probability')
ax1.set_title('Mean Synaptic Probability Over Time')
ax1.grid(True)
# Plot change in synaptic probability over time
ax2.plot(time_bins, mean_prob_changes, 'r-', linewidth=2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Change in Probability')
ax2.set_title('Mean Change in Synaptic Probability Over Time')
ax2.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
#res[31]['latency_extremum']

# Calculate synaptic probability over time
time_bins, probabilities, prob_changes = calculate_synaptic_probability(latency_after, median_latency,  time_bin_size=100000)

# Plot the results
plot_synaptic_probability(time_bins, probabilities, prob_changes)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

def calculate_synaptic_probability(latency, lag, time_bin_size=30):
    """
    Calculate synaptic probability over time bins.
    
    Parameters:
    - latency: Dictionary containing 'spike time' and other data
    - lag: The lag value
    - time_bin_size: Size of time bins in ms (default: 30)
    
    Returns:
    - time_bins: Array of time bin centers (in seconds)
    - probabilities: Array of synaptic probabilities for each time bin
    - prob_changes: Array of changes in probability between consecutive time bins
    """
    # Extract all spike times
    all_spike_times = latency['spike time']
    
    # Determine the range of time bins
    min_time = np.min(all_spike_times)
    max_time = np.max(all_spike_times)
    
    # Create time bins
    time_edges = np.arange(min_time, max_time + time_bin_size, time_bin_size)
    time_bins = (time_edges[:-1] + time_edges[1:]) / 2  # Centers of bins
    
    # Convert time bins from ms to seconds
    time_bins_seconds = time_bins / 1000.0
    
    probabilities = []
    
    # For each time bin, calculate synaptic probability
    for i in range(len(time_bins)):
        bin_start = time_edges[i]
        bin_end = time_edges[i+1]
        
        # Filter latency data for current time bin
        time_mask = (all_spike_times >= bin_start) & (all_spike_times < bin_end)
        bin_latency = latency[time_mask]
        
        if len(bin_latency) == 0:
            # No spikes in this bin
            probabilities.append(0)
            continue
        
        # Filter for output spikes
        data_output = bin_latency[bin_latency['category'] == 'output']
        
        # Filter for latency within lag ± 1 ms
        latency_filtered = data_output[(data_output['latency'] >= lag - 1) & 
                                     (data_output['latency'] <= lag + 1)]
        
        # Get input spikes in this bin
        input_spikes = bin_latency[bin_latency['category'] == 'input']
        input_spike_counts = len(input_spikes)
        
        if input_spike_counts == 0:
            # No input spikes in this bin
            probabilities.append(0)
            continue
        
        # Count output spikes that correspond to input spikes
        output_spike_counts = np.sum(np.isin(latency_filtered['input spike'], 
                                          input_spikes['input spike']))
        
        # Calculate probability
        probability = output_spike_counts / input_spike_counts
        probabilities.append(probability)
    
    probabilities = np.array(probabilities)
    
    # Calculate changes in probability
    prob_changes = np.diff(probabilities, prepend=probabilities[0])
    
    return time_bins_seconds, probabilities, prob_changes

# Approach 1: Using interpolation to align all time series to a common time axis
def interpolate_and_plot(data_te):
    all_time_bins = []
    all_probabilities = []
    all_prob_changes = []
    
    # First pass: collect all data
    for row in data_te['validated_results']:
        if row['validation'] == 'good':
            latency = row['latency_extremum']
            lag = row['lag']
            time_bins, probabilities, prob_changes = calculate_synaptic_probability(latency, lag, time_bin_size=25000)
            all_time_bins.append(time_bins)
            all_probabilities.append(probabilities)
            all_prob_changes.append(prob_changes)
    
    # Find global time range
    min_time = min([np.min(tb) for tb in all_time_bins])
    max_time = max([np.max(tb) for tb in all_time_bins])
    
    # Create a common time axis with a reasonable number of points
    num_points = 100  # Adjust as needed
    common_time_axis = np.linspace(min_time, max_time, num_points)
    
    # Interpolate each dataset to the common time axis
    interpolated_probs = []
    interpolated_changes = []
    
    for i in range(len(all_time_bins)):
        # Create interpolation functions
        f_prob = interp1d(all_time_bins[i], all_probabilities[i], 
                          bounds_error=False, fill_value=0)
        f_change = interp1d(all_time_bins[i], all_prob_changes[i], 
                           bounds_error=False, fill_value=0)
        
        # Interpolate to common time axis
        interpolated_probs.append(f_prob(common_time_axis))
        interpolated_changes.append(f_change(common_time_axis))
    
    # Convert to numpy arrays now that they all have the same shape
    interpolated_probs = np.array(interpolated_probs)
    interpolated_changes = np.array(interpolated_changes)
    
    # Calculate mean
    mean_probabilities = np.mean(interpolated_probs, axis=0)
    mean_prob_changes = np.mean(interpolated_changes, axis=0)
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot synaptic probability over time
    ax1.plot(common_time_axis, mean_probabilities, 'b-', linewidth=2)
    ax1.set_ylabel('Synaptic Probability')
    ax1.set_title('Mean Synaptic Probability Over Time')
    ax1.grid(True)
    
    # Plot change in synaptic probability over time
    ax2.plot(common_time_axis, mean_prob_changes, 'r-', linewidth=2)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Change in Probability')
    ax2.set_title('Mean Change in Synaptic Probability Over Time')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Approach 2: Alternative method using masking (useful if interpolation isn't appropriate)
def mask_and_plot(data_te):
    all_data = []
    
    # Collect all data with their time bins
    for row in data_te['validated_results']:
        if row['validation'] == 'good':
            latency = row['latency_extremum']
            lag = row['lag']
            time_bins, probabilities, prob_changes = calculate_synaptic_probability(latency, lag, time_bin_size=25000)
            all_data.append((time_bins, probabilities, prob_changes))
    
    # Find common time range
    min_time = max([np.min(d[0]) for d in all_data])  # Latest start time
    max_time = min([np.max(d[0]) for d in all_data])  # Earliest end time
    
    # Create time bins within the common range
    bin_size = 0.025  # 25ms in seconds
    common_bins = np.arange(min_time, max_time + bin_size, bin_size)
    
    # Initialize arrays for summing
    sum_probs = np.zeros(len(common_bins))
    sum_changes = np.zeros(len(common_bins))
    counts = np.zeros(len(common_bins))
    
    # Process each dataset
    for time_bins, probs, changes in all_data:
        # Find indices that fall within common range
        valid_indices = (time_bins >= min_time) & (time_bins <= max_time)
        
        # Get valid data
        valid_times = time_bins[valid_indices]
        valid_probs = probs[valid_indices]
        valid_changes = changes[valid_indices]
        
        # Assign to nearest bin in common_bins
        for t, p, c in zip(valid_times, valid_probs, valid_changes):
            # Find closest bin
            bin_idx = np.argmin(np.abs(common_bins - t))
            
            # Add to sums
            sum_probs[bin_idx] += p
            sum_changes[bin_idx] += c
            counts[bin_idx] += 1
    
    # Calculate means, avoiding division by zero
    mask = counts > 0
    mean_probs = np.zeros_like(sum_probs)
    mean_changes = np.zeros_like(sum_changes)
    
    mean_probs[mask] = sum_probs[mask] / counts[mask]
    mean_changes[mask] = sum_changes[mask] / counts[mask]
    
    # Plot results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Plot synaptic probability over time
    ax1.plot(common_bins, mean_probs, 'b-', linewidth=2)
    ax1.set_ylabel('Synaptic Probability')
    ax1.set_title('Mean Synaptic Probability Over Time')
    ax1.grid(True)
    
    # Plot change in synaptic probability over time
    ax2.plot(common_bins, mean_changes, 'r-', linewidth=2)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Change in Probability')
    ax2.set_title('Mean Change in Synaptic Probability Over Time')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()



In [ ]:
# Example usage:
interpolate_and_plot(data_te)  # Use this approach if interpolation is appropriate


In [ ]:
mask_and_plot(data_te)         # Use this approach if you prefer binning without interpolation